# Clone the Dataset Repository


In [ ]:
!git clone https://huggingface.co/datasets/billsum

Cloning into 'billsum'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 64 (delta 28), reused 16 (delta 16), pack-reused 28 (from 1)
Unpacking objects: 100% (64/64), 17.67 KiB | 754.00 KiB/s, done.
Filtering content: 100% (3/3), 108.46 MiB | 27.54 MiB/s, done.


# Install Necessary Libraries

In [ ]:
! pip install -q transformers[torch]

In [ ]:
!pip install transformers datasets rouge_score accelerate bitsandbytes

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=0d8daa8f0fff991ecb2b265a9c1564044eeebfbcb415685b582a1623eb7b69b3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's depende

# Load the Dataset


In [ ]:
from datasets import load_dataset
import pandas as pd
from datasets import Dataset
import re
import string

# Load the dataset from parquet files
train_df = pd.read_parquet('/content/billsum/data/train-00000-of-00001.parquet')
test_df = pd.read_parquet('/content/billsum/data/test-00000-of-00001.parquet')
ca_test_df = pd.read_parquet('/content/billsum/data/ca_test-00000-of-00001.parquet')

# Function to normalize text
def normalize_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

# Apply normalization
train_df['text'] = train_df['text'].apply(normalize_text)
train_df['summary'] = train_df['summary'].apply(normalize_text)
test_df['text'] = test_df['text'].apply(normalize_text)
test_df['summary'] = test_df['summary'].apply(normalize_text)
ca_test_df['text'] = ca_test_df['text'].apply(normalize_text)
ca_test_df['summary'] = ca_test_df['summary'].apply(normalize_text)

In [ ]:
# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
ca_test_dataset = Dataset.from_pandas(ca_test_df)

# Split the dataset
billsum = train_dataset.train_test_split(test_size=1)

# Tokenize the Dataset


In [ ]:
  import nltk
  nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Tokenize the dataset
from transformers import AutoTokenizer
from nltk.tokenize import sent_tokenize

tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Function for segmentation

tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Function for segmentation
def segment_text(text, max_len=512):
    sentences = sent_tokenize(text)
    segments = []
    current_segment = ""

    for sentence in sentences:
        sentence_tokenized_length = len(tokenizer(sentence)["input_ids"])

        # If the sentence alone is longer than max_len, truncate it
        if sentence_tokenized_length > max_len:
            truncated_sentence = tokenizer.decode(tokenizer(sentence, max_length=max_len, truncation=True)["input_ids"])
            segments.append(truncated_sentence)
        else:
            if len(tokenizer(current_segment + sentence)["input_ids"]) <= max_len:
                current_segment += sentence + " "
            else:
                segments.append(current_segment.strip())
                current_segment = sentence + " "

    if current_segment:
        segments.append(current_segment.strip())

    return segments

# Preprocessing function with normalization and segmentation
def preprocess_function(examples):
    all_model_inputs = []
    all_labels = []

    for doc, summary in zip(examples["text"], examples["summary"]):
        # Normalise text and summary
        doc = normalize_text(doc)
        summary = normalize_text(summary)

        # Segment the document
        segments = segment_text(doc)

        for segment in segments:
            inputs = "summarize: " + segment
            model_inputs = tokenizer(inputs, max_length=512, truncation=True)
            labels = tokenizer(summary, max_length=128, truncation=True)
            model_inputs["labels"] = labels["input_ids"]
            all_model_inputs.append(model_inputs)

    return {key: [d[key] for d in all_model_inputs] for key in all_model_inputs[0]}

# Tokenize the dataset
tokenized_billsum = billsum.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/3789 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2370 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/15160 [00:00<?, ? examples/s]

# Prepare Data Collator

In [ ]:
# Data collator
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.3 MB/s eta 0:00:00


# Define Evaluation Metrics

In [ ]:
# Evaluation metric
import evaluate

rouge = evaluate.load("rouge")
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

# Fine-Tune the Model


In [ ]:
# Fine-tuning the model
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
!pip install accelerate>=0.21.0

# Train the Model

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_fine_tuned_flan_t5_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("my_fine_tuned_flan_t5_model")

# Summarization example
from transformers import pipeline

text = test_df.iloc[100]['text']
text = "summarize: " + text

summarizer = pipeline("summarization", model="my_fine_tuned_flan_t5_model")
pred = summarizer(text)

# Display the summary
print(pred[0]['summary_text'])

# Evaluation
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_fine_tuned_flan_t5_model")
inputs = tokenizer(text, return_tensors="pt").input_ids

from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("my_fine_tuned_flan_t5_model")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)
summary_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Compute ROUGE score
preds = [summary_text]
labels = [test_df.iloc[100]['summary']]
rouge.compute(predictions=preds, references=labels, use_stemmer=True)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.585985,0.238400,0.166100,0.222200,0.222200,19.000000
2,No log,2.484938,0.247400,0.179000,0.232300,0.232300,19.000000
3,2.898200,2.445721,0.250000,0.182100,0.234800,0.234800,19.000000
4,2.898200,2.433619,0.250700,0.182900,0.235500,0.235400,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Token indices sequence length is longer than the specified maximum sequence length for this model (1579 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1577 > 5

afghanistan and central asian republics sustainable food production act of 2001 authorizes the administrator of the united states agency for international development to provide assistance in accordance with the provisions of this act to develop durable food production for the u.s. and the mountainous regions of central asia through restocking seed replacing breeding livestock restoring basic irrigation systems and providing access to credit for food production processing or marketing enterprises through rural microenterprise loans and v providing technical assistance to the national and regional governments of such countries


{'rouge1': 0.6214689265536724,
 'rouge2': 0.44571428571428573,
 'rougeL': 0.5423728813559322,
 'rougeLsum': 0.5423728813559322}